# 네이버 API에서 키워드를 입력받아 블록, 뉴스, 책 데이터 수집
## 각각의 카테고리별로 csv 파일로 저장
## 파일명에는 수집날짜가 자동으로 파일명에 포함되도록
## 반복되는 부분은 함수를 이용해서 중복 줄이기

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import re
from datetime import datetime
load_dotenv(dotenv_path="./data/.env")

True

In [2]:
def text_clean(text):
    # HTML 태그를 없애는 정규표현식
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    result = result.replace("  ", " ").replace("  ", " ").replace("  ", " ")
    return result

# 1페이지 수집

In [3]:
keyword = input("검색할 키워드를 입력해주세요: ")

검색할 키워드를 입력해주세요: 부동산


In [9]:
user_id = os.getenv("Id")
user_secret = os.getenv("Secret")
url = f"https://openapi.naver.com/v1/search/news"
payload = dict(query=keyword, display=100, start=1, sort="date")
headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
r = requests.get(url, params=payload, headers=headers)
response = r.json()

total_page = response['total'] // 100 + 1

result = {}
for item in response['items']:
#     print(item)
    for key, value in item.items():
        if key in ['title', 'description']:
            result.setdefault(key, []).append(text_clean(value))
        else:
            result.setdefault(key, []).append(value)
df = pd.DataFrame(result)
df

,title,originallink,link,description,pubDate
0,한은 연내 금리 동결 유력 부동산 환율 에 발목 잡혔다,http://www.newslock.co.kr/news/articleView.htm...,http://www.newslock.co.kr/news/articleView.htm...,들썩이는 부동산 시장과 1400원대의 불안한 환율 등 금융안정 리스크가 금리 인하의...,"Wed, 15 Oct 2025 14:48:00 +0900"
1,장동혁 부동산 대책 quot 청년 서민 죽이기 주택완박 quot,https://www.obsnews.co.kr/news/articleView.htm...,https://www.obsnews.co.kr/news/articleView.htm...,장동혁 국민의힘 대표는 정부가 주택담보대출 한도를 줄이는 등의 부동산 대책을 발표한...,"Wed, 15 Oct 2025 14:48:00 +0900"
2,10 15 부동산 대책 나오자 국토부 홈페이지 마비,https://magazine.hankyung.com/business/article...,https://n.news.naver.com/mnews/article/050/000...,10 15 부동산 대책 은 서울시 내 강남 서초 송파 용산과 일부 재건축 재개발 ...,"Wed, 15 Oct 2025 14:48:00 +0900"
3,9월 전국 집값 0 09 4개월 연속 오름세,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,15일 한국부동산원 전국 주택가격 동향 조사에 따르면 지난달 전국 주택가격은 전월 ...,"Wed, 15 Oct 2025 14:48:00 +0900"
4,신한투자증권 프리미어 MyPB 멤버스 관리자산 1조 돌파,https://www.hankyung.com/article/202510150289i,https://n.news.naver.com/mnews/article/015/000...,투자 상담 외에도 세무 부동산 은퇴설계 등 다양한 전문 콘텐츠와 프리미엄 혜택을 함...,"Wed, 15 Oct 2025 14:48:00 +0900"
...,...,...,...,...,...
95,우리은행 WON뱅킹서 아파트 하자 점검 서비스 제공,https://www.newsfreezone.co.kr/news/articleVie...,https://www.newsfreezone.co.kr/news/articleVie...,접속경로는 우리WON뱅킹 생활편의 WON하는 부동산 아파트 입주 전에는 꼭 홈체크 ...,"Wed, 15 Oct 2025 14:32:00 +0900"
96,2025 국감 최근 5년간 위반 부동산 광고 6만건 신고,https://www.asiatime.co.kr/article/20251015500210,https://www.asiatime.co.kr/article/20251015500210,허위 과장 부동산 광고에 대한 규정 위반 적발 건수가 매년 1만 건 이상 발생하고 ...,"Wed, 15 Oct 2025 14:32:00 +0900"
97,국민의힘 10 15 부동산대책에 quot 청년 서민 주택 완박 부동산 계엄 quot,https://www.pressian.com/pages/articles/202510...,https://n.news.naver.com/mnews/article/002/000...,quot 시장 메커니즘 이해 못한 더 센 규제 만 반복 quot 조국혁신당 quo...,"Wed, 15 Oct 2025 14:31:00 +0900"
98,9월 서울 집값 더 달아올랐다 한강벨트 급등,https://biz.sbs.co.kr/article_hub/20000265321?...,https://n.news.naver.com/mnews/article/374/000...,15일 한국부동산원이 발표한 9월 전국 주택가격 동향 조사 결과에 따르면 서울의 주...,"Wed, 15 Oct 2025 14:31:00 +0900"


In [22]:
import time

# 전체페이지 수집

In [23]:
keyword = input("검색할 키워드를 입력해 주세요: ")

blog_data = []
news_data = []
book_data = []

category = ['blog','news','book']

for cate in category:
    page_num = 1
    total_page = 1
    start_num = 1
    while page_num <= total_page:
        time.sleep(0.2)
        print(cate, page_num, total_page, end="\r")
        user_id = os.getenv("Id")
        user_secret = os.getenv("Secret")
        url = f"https://openapi.naver.com/v1/search/news"
        payload = dict(query=keyword, display=100, start=1, sort="date")
        headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
        r = requests.get(url, params=payload, headers=headers)
        response = r.json()
#         print(response)
        result = {}
        for item in response['items']:
            
            for key, value in item.items():
                if key in ['title','description']:
                    result.setdefault(key, []).append(text_clean(value))
                else:
                    result.setdefault(key, []).append(value)
        
        if cate == "blog":
            blog_data.append(pd.DataFrame(result))
        elif cate == "news":
            news_data.append(pd.DataFrame(result))
        elif cate == "book":
            book_data.append(pd.DataFrame(result))
            
        total_page = response['total'] // 100 + 1
        if total_page > 11:
            total_page = 11
        else:
            total_page = response['total'] // 100 + 1
        
        
        page_num += 1
        
        if start_num < 901:
            start_num += 100
        elif start_num > 900:
            start_num += 99

blog_df = pd.concat(blog_data)
news_df = pd.concat(news_data)
book_df = pd.concat(book_data)

blog_df = blog_df.reset_index(drop=True)
news_df = news_df.reset_index(drop=True)
book_df = book_df.reset_index(drop=True)

display(blog_df)
display(news_df)
display(book_df)

검색할 키워드를 입력해 주세요: 핀테크


,title,originallink,link,description,pubDate
0,핀산협 컴플라이언스 4 0 AI와 레그테크 융합 세미나 개최,https://www.etnews.com/20251015000273,https://n.news.naver.com/mnews/article/030/000...,한국핀테크산업협회가 15일 서울국제금융오피스에서 컴플라이언스 4 0 AI와 레그테크...,"Wed, 15 Oct 2025 15:05:00 +0900"
1,트립비토즈 두바이 GITEX 2025서 iOL과 전략적 파트너십 체결,http://www.enewstoday.co.kr/news/articleView.h...,http://www.enewstoday.co.kr/news/articleView.h...,iOL은 전 세계 호텔과 여행 바이어 간 연결을 간소화하는 iOL X 다국적 결제 ...,"Wed, 15 Oct 2025 14:56:00 +0900"
2,삼성카드 잇단 PLCC 출시로 시장 판도 재편한다,http://www.opinionnews.co.kr/news/articleView....,http://www.opinionnews.co.kr/news/articleView....,여기에 핀테크 인터넷은행 등 신규 사업자의 진입으로 결제시장이 포화되자 카드사들은 ...,"Wed, 15 Oct 2025 14:54:00 +0900"
3,안랩 동남아 보안시장 공략 박차 인니 말레이 현지 수요 확인,https://www.tokenpost.kr/news/tech/294623,https://www.tokenpost.kr/news/tech/294623,이처럼 정보보안과 핀테크 분야에서 나타나는 기업 간 협력과 해외 진출은 국내 기술과...,"Wed, 15 Oct 2025 14:52:00 +0900"
4,PFCT 호주서 AI 리스크 관리 노하우 전수한다,https://www.etoday.co.kr/news/view/2514976,https://www.etoday.co.kr/news/view/2514976,이투데이 김은재 기자 핀테크 기업 피에프씨테크놀로지스 PFCT 는 이수환 대표가 올...,"Wed, 15 Oct 2025 14:48:00 +0900"
...,...,...,...,...,...
1095,미래에셋증권 연금 50조 해외주식 50조 돌파 NH투자증권 글로벌 사모,http://www.sisaon.co.kr/news/articleView.html?...,http://www.sisaon.co.kr/news/articleView.html?...,올해 카카오페이증권은 AI에 기반한 투자정보 서비스를 순차적으로 선보이는 등 핀테크...,"Tue, 14 Oct 2025 18:02:00 +0900"
1096,40일 뒤 받은 통신 계좌정보 이래서 범인 잡겠나,https://www.mk.co.kr/article/11441313,https://n.news.naver.com/mnews/article/009/000...,14일 국회 행정안전위원회 소속 위성곤 더불어민주당 의원실이 서울청 부산청 경기남부...,"Tue, 14 Oct 2025 18:02:00 +0900"
1097,재개약률 94 공유오피스 패스트파이브 입주사 사로잡은 비결,https://www.mt.co.kr/future/2025/10/14/2025101...,https://n.news.naver.com/mnews/article/008/000...,장기 입주사인 핀테크 기업 와이어바알리는 2019년 첫 입주 후 6년 이상 패스트파...,"Tue, 14 Oct 2025 18:00:00 +0900"
1098,코인으로 동문회비를 연세대 총동문회 가상자산 납부 도입,https://www.yna.co.kr/view/AKR2025101416530000...,https://n.news.naver.com/mnews/article/001/001...,외화 송금이 불편하다는 해외 동문 등 여러 의견을 수렴한 총동문회는 핀테크 금융기술...,"Tue, 14 Oct 2025 17:55:00 +0900"


,title,originallink,link,description,pubDate
0,핀산협 컴플라이언스 4 0 AI와 레그테크 융합 세미나 개최,https://www.etnews.com/20251015000273,https://n.news.naver.com/mnews/article/030/000...,한국핀테크산업협회가 15일 서울국제금융오피스에서 컴플라이언스 4 0 AI와 레그테크...,"Wed, 15 Oct 2025 15:05:00 +0900"
1,트립비토즈 두바이 GITEX 2025서 iOL과 전략적 파트너십 체결,http://www.enewstoday.co.kr/news/articleView.h...,http://www.enewstoday.co.kr/news/articleView.h...,iOL은 전 세계 호텔과 여행 바이어 간 연결을 간소화하는 iOL X 다국적 결제 ...,"Wed, 15 Oct 2025 14:56:00 +0900"
2,삼성카드 잇단 PLCC 출시로 시장 판도 재편한다,http://www.opinionnews.co.kr/news/articleView....,http://www.opinionnews.co.kr/news/articleView....,여기에 핀테크 인터넷은행 등 신규 사업자의 진입으로 결제시장이 포화되자 카드사들은 ...,"Wed, 15 Oct 2025 14:54:00 +0900"
3,안랩 동남아 보안시장 공략 박차 인니 말레이 현지 수요 확인,https://www.tokenpost.kr/news/tech/294623,https://www.tokenpost.kr/news/tech/294623,이처럼 정보보안과 핀테크 분야에서 나타나는 기업 간 협력과 해외 진출은 국내 기술과...,"Wed, 15 Oct 2025 14:52:00 +0900"
4,PFCT 호주서 AI 리스크 관리 노하우 전수한다,https://www.etoday.co.kr/news/view/2514976,https://www.etoday.co.kr/news/view/2514976,이투데이 김은재 기자 핀테크 기업 피에프씨테크놀로지스 PFCT 는 이수환 대표가 올...,"Wed, 15 Oct 2025 14:48:00 +0900"
...,...,...,...,...,...
1095,미래에셋증권 연금 50조 해외주식 50조 돌파 NH투자증권 글로벌 사모,http://www.sisaon.co.kr/news/articleView.html?...,http://www.sisaon.co.kr/news/articleView.html?...,올해 카카오페이증권은 AI에 기반한 투자정보 서비스를 순차적으로 선보이는 등 핀테크...,"Tue, 14 Oct 2025 18:02:00 +0900"
1096,40일 뒤 받은 통신 계좌정보 이래서 범인 잡겠나,https://www.mk.co.kr/article/11441313,https://n.news.naver.com/mnews/article/009/000...,14일 국회 행정안전위원회 소속 위성곤 더불어민주당 의원실이 서울청 부산청 경기남부...,"Tue, 14 Oct 2025 18:02:00 +0900"
1097,재개약률 94 공유오피스 패스트파이브 입주사 사로잡은 비결,https://www.mt.co.kr/future/2025/10/14/2025101...,https://n.news.naver.com/mnews/article/008/000...,장기 입주사인 핀테크 기업 와이어바알리는 2019년 첫 입주 후 6년 이상 패스트파...,"Tue, 14 Oct 2025 18:00:00 +0900"
1098,코인으로 동문회비를 연세대 총동문회 가상자산 납부 도입,https://www.yna.co.kr/view/AKR2025101416530000...,https://n.news.naver.com/mnews/article/001/001...,외화 송금이 불편하다는 해외 동문 등 여러 의견을 수렴한 총동문회는 핀테크 금융기술...,"Tue, 14 Oct 2025 17:55:00 +0900"


,title,originallink,link,description,pubDate
0,핀산협 컴플라이언스 4 0 AI와 레그테크 융합 세미나 개최,https://www.etnews.com/20251015000273,https://n.news.naver.com/mnews/article/030/000...,한국핀테크산업협회가 15일 서울국제금융오피스에서 컴플라이언스 4 0 AI와 레그테크...,"Wed, 15 Oct 2025 15:05:00 +0900"
1,트립비토즈 두바이 GITEX 2025서 iOL과 전략적 파트너십 체결,http://www.enewstoday.co.kr/news/articleView.h...,http://www.enewstoday.co.kr/news/articleView.h...,iOL은 전 세계 호텔과 여행 바이어 간 연결을 간소화하는 iOL X 다국적 결제 ...,"Wed, 15 Oct 2025 14:56:00 +0900"
2,삼성카드 잇단 PLCC 출시로 시장 판도 재편한다,http://www.opinionnews.co.kr/news/articleView....,http://www.opinionnews.co.kr/news/articleView....,여기에 핀테크 인터넷은행 등 신규 사업자의 진입으로 결제시장이 포화되자 카드사들은 ...,"Wed, 15 Oct 2025 14:54:00 +0900"
3,안랩 동남아 보안시장 공략 박차 인니 말레이 현지 수요 확인,https://www.tokenpost.kr/news/tech/294623,https://www.tokenpost.kr/news/tech/294623,이처럼 정보보안과 핀테크 분야에서 나타나는 기업 간 협력과 해외 진출은 국내 기술과...,"Wed, 15 Oct 2025 14:52:00 +0900"
4,PFCT 호주서 AI 리스크 관리 노하우 전수한다,https://www.etoday.co.kr/news/view/2514976,https://www.etoday.co.kr/news/view/2514976,이투데이 김은재 기자 핀테크 기업 피에프씨테크놀로지스 PFCT 는 이수환 대표가 올...,"Wed, 15 Oct 2025 14:48:00 +0900"
...,...,...,...,...,...
1095,미래에셋증권 연금 50조 해외주식 50조 돌파 NH투자증권 글로벌 사모,http://www.sisaon.co.kr/news/articleView.html?...,http://www.sisaon.co.kr/news/articleView.html?...,올해 카카오페이증권은 AI에 기반한 투자정보 서비스를 순차적으로 선보이는 등 핀테크...,"Tue, 14 Oct 2025 18:02:00 +0900"
1096,40일 뒤 받은 통신 계좌정보 이래서 범인 잡겠나,https://www.mk.co.kr/article/11441313,https://n.news.naver.com/mnews/article/009/000...,14일 국회 행정안전위원회 소속 위성곤 더불어민주당 의원실이 서울청 부산청 경기남부...,"Tue, 14 Oct 2025 18:02:00 +0900"
1097,재개약률 94 공유오피스 패스트파이브 입주사 사로잡은 비결,https://www.mt.co.kr/future/2025/10/14/2025101...,https://n.news.naver.com/mnews/article/008/000...,장기 입주사인 핀테크 기업 와이어바알리는 2019년 첫 입주 후 6년 이상 패스트파...,"Tue, 14 Oct 2025 18:00:00 +0900"
1098,코인으로 동문회비를 연세대 총동문회 가상자산 납부 도입,https://www.yna.co.kr/view/AKR2025101416530000...,https://n.news.naver.com/mnews/article/001/001...,외화 송금이 불편하다는 해외 동문 등 여러 의견을 수렴한 총동문회는 핀테크 금융기술...,"Tue, 14 Oct 2025 17:55:00 +0900"


In [21]:
print(response)

{'errorMessage': 'Rate limit exceeded. (속도 제한을 초과했습니다.)', 'errorCode': '012'}


In [43]:
news_df

,title,originallink,link,description,pubDate
0,핀산협 컴플라이언스 4 0 AI와 레그테크 융합 세미나 개최,https://www.etnews.com/20251015000273,https://n.news.naver.com/mnews/article/030/000...,한국핀테크산업협회가 15일 서울국제금융오피스에서 컴플라이언스 4 0 AI와 레그테크...,"Wed, 15 Oct 2025 15:05:00 +0900"
1,트립비토즈 두바이 GITEX 2025서 iOL과 전략적 파트너십 체결,http://www.enewstoday.co.kr/news/articleView.h...,http://www.enewstoday.co.kr/news/articleView.h...,iOL은 전 세계 호텔과 여행 바이어 간 연결을 간소화하는 iOL X 다국적 결제 ...,"Wed, 15 Oct 2025 14:56:00 +0900"
2,삼성카드 잇단 PLCC 출시로 시장 판도 재편한다,http://www.opinionnews.co.kr/news/articleView....,http://www.opinionnews.co.kr/news/articleView....,여기에 핀테크 인터넷은행 등 신규 사업자의 진입으로 결제시장이 포화되자 카드사들은 ...,"Wed, 15 Oct 2025 14:54:00 +0900"
3,안랩 동남아 보안시장 공략 박차 인니 말레이 현지 수요 확인,https://www.tokenpost.kr/news/tech/294623,https://www.tokenpost.kr/news/tech/294623,이처럼 정보보안과 핀테크 분야에서 나타나는 기업 간 협력과 해외 진출은 국내 기술과...,"Wed, 15 Oct 2025 14:52:00 +0900"
4,PFCT 호주서 AI 리스크 관리 노하우 전수한다,https://www.etoday.co.kr/news/view/2514976,https://www.etoday.co.kr/news/view/2514976,이투데이 김은재 기자 핀테크 기업 피에프씨테크놀로지스 PFCT 는 이수환 대표가 올...,"Wed, 15 Oct 2025 14:48:00 +0900"
...,...,...,...,...,...
1095,미래에셋증권 연금 50조 해외주식 50조 돌파 NH투자증권 글로벌 사모,http://www.sisaon.co.kr/news/articleView.html?...,http://www.sisaon.co.kr/news/articleView.html?...,올해 카카오페이증권은 AI에 기반한 투자정보 서비스를 순차적으로 선보이는 등 핀테크...,"Tue, 14 Oct 2025 18:02:00 +0900"
1096,40일 뒤 받은 통신 계좌정보 이래서 범인 잡겠나,https://www.mk.co.kr/article/11441313,https://n.news.naver.com/mnews/article/009/000...,14일 국회 행정안전위원회 소속 위성곤 더불어민주당 의원실이 서울청 부산청 경기남부...,"Tue, 14 Oct 2025 18:02:00 +0900"
1097,재개약률 94 공유오피스 패스트파이브 입주사 사로잡은 비결,https://www.mt.co.kr/future/2025/10/14/2025101...,https://n.news.naver.com/mnews/article/008/000...,장기 입주사인 핀테크 기업 와이어바알리는 2019년 첫 입주 후 6년 이상 패스트파...,"Tue, 14 Oct 2025 18:00:00 +0900"
1098,코인으로 동문회비를 연세대 총동문회 가상자산 납부 도입,https://www.yna.co.kr/view/AKR2025101416530000...,https://n.news.naver.com/mnews/article/001/001...,외화 송금이 불편하다는 해외 동문 등 여러 의견을 수렴한 총동문회는 핀테크 금융기술...,"Tue, 14 Oct 2025 17:55:00 +0900"


## 코드를 함수화

In [41]:
def naver_api(keyword, cate):
    page_num = 1
    total_page = 1
    start_num = 1
    
    all_data = []
    while page_num <= total_page:
        print(cate, page_num, total_page, end="\r")
        time.sleep(0.8)
        user_id = os.getenv("Id")
        user_secret = os.getenv("Secret")
        url = f"https://openapi.naver.com/v1/search/{cate}"
        payload = dict(query=keyword, display=100, start=start_num, sort="date")
        headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
        r = requests.get(url, params=payload, headers=headers)
        response = r.json()

        result = {}
        for item in response['items']:

            for key, value in item.items():
                if key in ['title', 'description']:
                    result.setdefault(key, []).append(text_clean(value))
                else:
                    result.setdefault(key, []).append(value)
        
        all_data.append(pd.DataFrame(result))
        
        total_page = response['total'] // 100 + 1
        if total_page > 11:
            total_page = 11
        else:
            total_page = response['total'] // 100 + 1


        page_num += 1

        if start_num < 901:
            start_num += 100
        elif start_num > 900:
            start_num += 99

    
    result_df = pd.concat(all_data)
    result_df = result_df.reset_index(drop=True)
    return result_df 

In [38]:
today = datetime.today()
today

datetime.datetime(2025, 10, 15, 15, 23, 54, 161202)

In [39]:
date = f"{today.year}{today.month:02d}{today.day:02d}"
date

'20251015'

In [42]:
keyword = input("검색어를 입력하세요: ")
date = f"{today.year}{today.month:02d}{today.day:02d}"
for cate in category:
    result = naver_api(keyword, cate)
    result.to_csv(f"./data/{keyword}_{cate}_{date}.csv", index=False, encoding="utf-8-sig")

검색어를 입력하세요: 핀테크
